# Analysis of Sioux Falls UAM MATSim Scenario

## Includes

In [2]:
import matsim
import pandas as pd
from collections import defaultdict
from xml.etree.ElementTree import Element
from sqlalchemy import create_engine, text
%matplotlib inline

## List of unique modes

In [2]:
plans = matsim.plan_reader('scenarios/sioux_falls_uam/output/output_plans.xml.gz', selectedPlansOnly = False)

# Each plan is returned as a tuple with its owning person (for now, is this ok?)
# - The name of the element is in its .tag (e.g. 'plan', 'leg', 'route', 'attributes')
# - An element's attributes are accessed using .attrib['attrib-name']
# - Use the element's .text field to get data outside of attributes (e.g. a route's list of links)
# - Every element can be iterated on to get its children (e.g. the plan's activities and legs)
# - Emits person even if that person has no plans

distinct_legs = set(())

for person, plan in plans:

    # do stuff with this plan, e.g.
    legs = filter(
        lambda e: e.tag == 'leg', plan)

    distinct_legs_individual = set(map(lambda a: a.get('mode'), legs))
    #print(person.attrib['id'], distinct_legs_individual)
    distinct_legs = distinct_legs.union(distinct_legs_individual)
print(distinct_legs)

{'pt', 'car', 'walk'}


## Obtain list of persons with home locations

In [3]:
import geoalchemy2
import geopandas as gpd
import shapely.geometry as shp

crs = {'init': "EPSG:26914"}

plans = matsim.plan_reader('scenarios/sioux_falls_uam/output/output_plans.xml.gz', selectedPlansOnly = True)

persons_list = []

for person, plan in plans:
    person_id = person.attrib['id']
    person_dict = {'person_id': person_id}
    
    
    # get x and y coordinates from first home activity
    for event in plan:
        if event.tag == 'activity' and event.attrib['type'] == 'home':
            home_x = float(event.attrib['x'])
            home_y = float(event.attrib['y'])
            break
    
    home_geom = shp.Point(home_x, home_y)
    
    
    person_dict.update({'home_x': home_x, 'home_y': home_y, 'home_geom': home_geom})
    
    
    persons_list.append(person_dict)
    
print(persons_list[0:100])    

[{'person_id': '10000_1', 'home_x': 686346.1102999998, 'home_y': 4824405.8913, 'home_geom': <shapely.geometry.point.Point object at 0x000002665A339640>}, {'person_id': '10001_1', 'home_x': 686463.2969000004, 'home_y': 4824239.2903, 'home_geom': <shapely.geometry.point.Point object at 0x000002665A339040>}, {'person_id': '10002_1', 'home_x': 686346.1102999998, 'home_y': 4824405.8913, 'home_geom': <shapely.geometry.point.Point object at 0x000002665A310970>}, {'person_id': '10003_1', 'home_x': 686457.5027000001, 'home_y': 4824397.363, 'home_geom': <shapely.geometry.point.Point object at 0x000002665A310A00>}, {'person_id': '10004_1', 'home_x': 686664.9104000004, 'home_y': 4824589.509299999, 'home_geom': <shapely.geometry.point.Point object at 0x000002665A3104C0>}, {'person_id': '10005_1', 'home_x': 686738.5410000002, 'home_y': 4824574.2083, 'home_geom': <shapely.geometry.point.Point object at 0x000002665A310BB0>}, {'person_id': '10006_1', 'home_x': 686457.5027000001, 'home_y': 4824397.363, 

## List of unique daily activity chains

In [13]:
plans = matsim.plan_reader('Outputs/output_20220510/ITERS/it.30/30.plans.xml.gz', selectedPlansOnly = False)

distinct_activity_lists = set(())

for person, plan in plans:
    
    
    # do stuff with this plan, e.g.
    activities = filter(
        lambda e: e.tag == 'activity', plan)
    
    
    activity_types = tuple(map(lambda a: a.get('type'), activities))
    activity_types_filtered = filter(lambda t: t != 'pt interaction', activity_types)
    activities_tuple = tuple(activity_types_filtered)
    if activities_tuple not in distinct_activity_lists:
        distinct_activity_lists.add(activities_tuple)
print(distinct_activity_lists)

{('home', 'secondary', 'home'), ('home', 'uam_interaction', 'uam_interaction', 'work', 'uam_interaction', 'uam_interaction', 'home'), ('home', 'uam_interaction', 'uam_interaction', 'secondary', 'uam_interaction', 'uam_interaction', 'home'), ('home', 'work', 'home')}
